# El formato de chat
Em este cuaderno, explorarás cómo puedes utilizar el formato chat para tener conversaciones extendidas con chatbots personalizados o especializados para tareas o comportamientos específicos.

## Setup


In [1]:
import openai
import textwrap # Para quebrar o texto em lineas
openai.api_key = "TU API KEY"


In [29]:
"""
Este código define dos funciones para generar respuestas de chat de OpenAI. La primera función, get_completion,
toma como entradas una pregunta y un modelo opcional, genera una única respuesta del modelo proporcionado y devuelve
el contenido del mensaje generado. La segunda función, get_completion_from_messages, toma una lista de mensajes, un
modelo opcional y un valor de temperatura opcional como entradas, genera una respuesta basada en los mensajes
proporcionados y devuelve el contenido del mensaje generado. La API de completado de chat de OpenAI se utiliza para
generar estos mensajes y el grado de aleatoriedad de los mensajes generados se controla mediante el parámetro de
temperatura.
"""
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [30]:
messages = [
    {"role":"system", "content":"Eres un asistente que habla como shakesperare"},
    {"role":"user", "content":"Dime una broma"},
    {"role":"assistant", "content":"¿Porque el pollo cruzó la carretera?"},
    {"role":"user", "content":"No lo se"}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "Porque quer\u00eda llegar al otro lado del camino, en un intento desafiante de completar su misi\u00f3n en busca de la felicidad y la libertad en un mundo lleno de obst\u00e1culos y desaf\u00edos impredecibles... O simplemente para escapar de la olla donde lo cocinar\u00edan. \u00a1Ja, ja, ja!",
  "role": "assistant"
}
Porque quería llegar al otro lado del camino, en un intento desafiante de completar su misión en busca de la felicidad y la libertad en un mundo lleno de obstáculos y desafíos impredecibles... O simplemente para escapar de la olla donde lo cocinarían. ¡Ja, ja, ja!


In [31]:
# Conversación con el chatbot sin contexto
messages = [
    {"role":"system", "content":"Eres un chatbot amigable"},
    {"role":"user", "content":"Hola, mi nombre es Juan"}
    ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "\u00a1Hola Juan! Es un gusto conversar contigo. \u00bfEn qu\u00e9 puedo ayudarte el d\u00eda de hoy?",
  "role": "assistant"
}
¡Hola Juan! Es un gusto conversar contigo. ¿En qué puedo ayudarte el día de hoy?


In [32]:
# No recuerda el nombre porque no hay contexto
messages = [
    {"role":"system", "content":"Eres un chatbot amigable"},
    {"role":"user", "content":"Si, ¿Puedes recordarme mi nombre?"}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "Por supuesto, \u00bfMe lo podr\u00edas decir?",
  "role": "assistant"
}
Por supuesto, ¿Me lo podrías decir?


In [33]:
messages = [
    {"role":"system", "content":"Eres un chatbot amigable"},
    {"role":"user", "content":"Hola, mi nombre es Juan"},
    {"role":"assistant", "content":"Hola Juan, ¿Té puedo ayudar en algo hoy?"},
    {"role":"user", "content":"Si, tienes recordar mi nombre"}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "S\u00ed, Juan, ya recuerdo tu nombre. \u00bfHay algo m\u00e1s en lo que pueda ayudarte?",
  "role": "assistant"
}
Sí, Juan, ya recuerdo tu nombre. ¿Hay algo más en lo que pueda ayudarte?


# OrderBot
Vamos a automatizar las respuestas del asistente para dar información a un cliente sobre las pizzas

In [34]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)
    
    

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [
    {"role":"system", "content":"""
     Eres OrderBot un servicio automatizado de recogida de pedidos de una pizzería.
     Primero saludas al cliente, luego recoges el pedido y después le preguntas si es para recoger \
     o para llevar.
     Espera para recoger todo el pedido. Luego lo resumes y comprueba si el cliente quiere añadir algo más.
     Si es una entrega a domicilio, pide la dirección.
     Asegúrate de aclarar todas las opciones, ingredientes extras y tamaños para identificar de forma única el \
     el artículo que el cliente quiere.
     
     Responde de manera corta y de forma amigable
     
     El menu incluye \
     Pizza peperoni 12.95, 10.00, 7.00 \
     Pizza de queso 10.95, 9.25, 6.50 \
     Pizza de berenjena 11.95, 9.25, 6.50 \
     Patatas fritas 3.95, 2.95 \
     Ensalada griega 7.95 \
     Extras: \
     Extra queso 1.00 \
     Champiñones 1.50 \
     Salchichas 1.50 \
     Beiicon 3.50 \
     Salsa de IA 0.50 \
     Pimientos 1.00 \
     Bebidas: \
     Coca Cola 3.50, 2.00, 1.00 \
     Fanta 3.50, 2.00, 1.00 \
     Botella de agua 2.00 \
    """}
]
     
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard                         
         
     


In [36]:
# Crear un JSON con el contenido de la conversación
messages = context.copy()
messages.append({'role':'system', 'content': 'Crea un resumen json del pedido de comida anterior. Detalla el precio de cada artículo. \
Los campos deben ser 1) pizza, incluir su tamaño, 2) Lista de ingredientes 3 Listas de bebidas, incluir el tamaño 4) Lista de los extras \
incluyendo su tamaño 5) Precio total'})

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "content": "Aqu\u00ed est\u00e1 el resumen JSON del pedido de comida anterior:\n\n```\n{\n  \"pizza\": {\n    \"tipo\": \"queso\",\n    \"tama\u00f1o\": \"grande\",\n    \"precio\": 10.95\n  },\n  \"ingredientes\": [\n    \"ninguno\"\n  ],\n  \"bebidas\": [\n    {\n      \"tipo\": \"Coca Cola\",\n      \"tama\u00f1o\": \"grande\",\n      \"precio\": 3.50\n    }\n  ],\n  \"extras\": [\n    {\n      \"tipo\": \"patatas fritas\",\n      \"tama\u00f1o\": \"peque\u00f1o\",\n      \"precio\": 2.95\n    }\n  ],\n  \"precio_total\": 16.40\n}\n``` \n\nEl precio total se calcula sumando el precio de la pizza, el precio de los extras y el precio de la bebida.",
  "role": "assistant"
}
Aquí está el resumen JSON del pedido de comida anterior:

```
{
  "pizza": {
    "tipo": "queso",
    "tamaño": "grande",
    "precio": 10.95
  },
  "ingredientes": [
    "ninguno"
  ],
  "bebidas": [
    {
      "tipo": "Coca Cola",
      "tamaño": "grande",
      "precio": 3.50
    }
  ],
  "extras": [
    {
 

In [37]:
html = response
prompt = f"Limpia el texto para que solo quede el JSON y crea una tabla html con el contenido del JSON del siguiete texto: {html}"
response1 = get_completion(messages)
print(response1)


InvalidRequestError: [{'role': 'system', 'content': '\n     Eres OrderBot un servicio automatizado de recogida de pedidos de una pizzería.\n     Primero saludas al cliente, luego recoges el pedido y después le preguntas si es para recoger      o para llevar.\n     Espera para recoger todo el pedido. Luego lo resumes y comprueba si el cliente quiere añadir algo más.\n     Si es una entrega a domicilio, pide la dirección.\n     Asegúrate de aclarar todas las opciones, ingredientes extras y tamaños para identificar de forma única el      el artículo que el cliente quiere.\n     \n     Responde de manera corta y de forma amigable\n     \n     El menu incluye      Pizza peperoni 12.95, 10.00, 7.00      Pizza de queso 10.95, 9.25, 6.50      Pizza de berenjena 11.95, 9.25, 6.50      Patatas fritas 3.95, 2.95      Ensalada griega 7.95      Extras:      Extra queso 1.00      Champiñones 1.50      Salchichas 1.50      Beiicon 3.50      Salsa de IA 0.50      Pimientos 1.00      Bebidas:      Coca